In [10]:
from route_parameters import *

In [14]:
test = Route_parameters([[4, 5, 6],[1, 2, 3],[1, 1, 3]],(1, 1), (1, 2))
print(test.get_distance())
print(test.min_speed())
print(test.get_time())

60.03141912169564
[1 1 3]
36.018851473017385


In [12]:
test = Route_parameters([4, 5, 6],(2, 4), (1, 7))
print(test.get_distance())
print(test.min_speed())
print(test.get_time())

189.80419291344214
[4 5 6]
37.960838582688424
